In [4]:
import pm4py
from tokenreplay import TokenReplay
from comparison import ModelComparator
from practical.ProcessMining.group1.shared.utils import event_log_to_dataframe, read_txt_test_logs

In [5]:
def compare_all_logs():
    logs = read_txt_test_logs("../shared/example_files/simple_event_logs_modified.txt")

    results_dict = {}

    for log_key, log in logs.items():
        # print(f"Processing log: {log_key}")
        log_df = event_log_to_dataframe(log)
        # print('log_df =====', log_df)
        log_df = pm4py.format_dataframe(log_df, case_id='case_id', activity_key='activity', timestamp_key='timestamp')

        # alpha
        net_alpha, im_alpha, fm_alpha = pm4py.discover_petri_net_alpha(log_df)

        # heuristic
        net_heuristic, im_heuristic, fm_heuristic = pm4py.discover_petri_net_heuristics(log_df)

        # im
        net_inductive, im_inductive, fm_inductive = pm4py.discover_petri_net_inductive(log_df)

        # #inductive inf
        net_inductive_inf, im_inductive_inf, fm_inductive_inf = pm4py.discover_petri_net_inductive(log_df,noise_threshold=0.7)

        m1 = TokenReplay(log_df, net_alpha, im_alpha, fm_alpha, "alpha")
        m2 = TokenReplay(log_df, net_heuristic, im_heuristic, fm_heuristic, "heuristic")
        m3 = TokenReplay(log_df, net_inductive, im_inductive, fm_inductive, "inductive")
        m4 = TokenReplay(log_df, net_inductive_inf, im_inductive_inf, fm_inductive_inf, "inductive_imf")

        comparator = ModelComparator([m1, m2, m3, m4])

        # print("===== all models values for log:", log_key, "=====")
        models_values = comparator.get_models_values()
        print('models_values ======', models_values)
        # print("=============================\n")

        print("===== Pareto Efficient Models for log:", log_key, "=====")
        results = comparator.run(x_dimension='f', y_dimension='s')
        # results_dict[log_key] = results
        print(results)
        print("=============================\n")

    # Print or return all results after processing all logs
    # for log_key, results in results_dict.items():
        # print(f"Results for log {log_key}:")
        # print(results)
        # print("=============================\n")

In [6]:
compare_all_logs()

replaying log with TBR, completed traces ::   0%|          | 0/3 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/6 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/3 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/3 [00:00<?, ?it/s]

{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')], 'reached_marking': ['end:1'], 'enabled_transitions_in_marking': set(), 'transitions_with_problems': [], 'missing_tokens': 0, 'consumed_tokens': 6, 'remaining_tokens': 0, 'produced_tokens': 6}
{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')], 'reached_marking': ['end:1'], 'enabled_transitions_in_marking': set(), 'transitions_with_problems': [], 'missing_tokens': 0, 'consumed_tokens': 6, 'remaining_tokens': 0, 'produced_tokens': 6}
{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(a, 'a'), (b, 'b'), (c, 'c'), (d, 'd')], 'reached_marking': ['end:1'], 'enabled_transitions_in_marking': set(), 'transitions_with_problems': [], 'missing_tokens': 0, 'consumed_tokens': 6, 'remaining_tokens': 0, 'produced_tokens': 6}
{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(a, 'a'), (c, 'c'

...

replaying log with TBR, completed traces ::   0%|          | 0/4 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/9 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/4 [00:00<?, ?it/s]

replaying log with TBR, completed traces ::   0%|          | 0/4 [00:00<?, ?it/s]

{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(5ebdac58-b574-46a9-84c3-55bd44cce8d9, 'a'), (c5ed4811-39ed-43f4-b842-cfcd2b0c29e9, 'b'), (d9639e18-5468-47bf-a96a-48867f81eee4, 'c'), (5e4f24e1-1b22-4c68-bd8b-3bc55694b3e5, 'd'), (76abf0fd-17af-46a1-887a-f1d637276643, 'f'), (a3da0cec-1204-4ceb-b5ca-2c8f1c977434, 'e')], 'reached_marking': ['sink:1'], 'enabled_transitions_in_marking': set(), 'transitions_with_problems': [], 'missing_tokens': 0, 'consumed_tokens': 8, 'remaining_tokens': 0, 'produced_tokens': 8}
{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(5ebdac58-b574-46a9-84c3-55bd44cce8d9, 'a'), (d9639e18-5468-47bf-a96a-48867f81eee4, 'c'), (c5ed4811-39ed-43f4-b842-cfcd2b0c29e9, 'b'), (5e4f24e1-1b22-4c68-bd8b-3bc55694b3e5, 'd'), (76abf0fd-17af-46a1-887a-f1d637276643, 'f'), (a3da0cec-1204-4ceb-b5ca-2c8f1c977434, 'e')], 'reached_marking': ['sink:1'], 'enabled_transitions_in_marking': set(), 'transitions_with_problems': [], 'missing_tokens': 